# 1. Introdução

Neste laboratório srá demonstrado o uso de manipulações geométricas para aummento de dataset.

In [ ]:
#pacotes e funções auxiliares
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from google.colab.patches import cv2_imshow
import math

def show_image(img, title):
  plt.figure(figsize=(10,5))
  plt.imshow(img)
  plt.title(title)
  plt.axis('off')
  plt.show()


# 2. Dados

In [ ]:
# Verifica se já foram baixadas as imagens do drive, baixando-as e descompactando se necessário
! [ ! -d "/content/cats_vs_dogs" ] && gdown -O /content/cats_vs_dogs.zip "1VuxwfbLZDXPt_0yKTauh4LRIhqMX1SdR" &&  unzip -q /content/cats_vs_dogs.zip -d /content && rm /content/cats_vs_dogs.zip

base_path = Path("/content/cats_vs_dogs")

In [ ]:
cats = [cv2.imread(str(p)) for p in (base_path/'train'/'cat').glob('000*.jpg')]

print(f'Imagens de gatos: {len(cats)}')

# 3. Multiplicação das Imagens

In [ ]:
# escolhe aleatorimente uma imagem
i = np.random.randint(0, len(cats))
image = cv2.cvtColor(cats[i], cv2.COLOR_BGR2RGB)
show_image(image, 'Original')

h, w = image.shape[:2]

augmented_images = []

for i in range(8):  # Gerar 8 imagens diferentes

    # Rotação + Escala
    angle = np.random.uniform(-30, 30)  # graus
    scale = np.random.uniform(0.8, 1.2)  # zoom
    M_rot = cv2.getRotationMatrix2D((w/2, h/2), angle, scale)

    # Cisalhamento
    kx = np.random.uniform(-0.3, 0.3)
    ky = np.random.uniform(-0.3, 0.3)
    M_shear = np.array([
        [1, kx, 0],
        [ky, 1, 0]
    ], dtype=np.float32)

    # Combinar Rotação + Cisalhamento
    M_combined = M_shear @ np.vstack([M_rot, [0, 0, 1]])  # vira 3x3

    # Translação aleatória
    tx = np.random.randint(-40, 40)
    ty = np.random.randint(-40, 40)
    M_combined[0, 2] += tx
    M_combined[1, 2] += ty

    # Perspectiva (Homografia)
    margin = 60  # controle da intensidade da perspectiva
    src_pts = np.float32([
        [0, 0],
        [w, 0],
        [0, h],
        [w, h]
    ])

    dst_pts = src_pts + np.random.uniform(-margin, margin, src_pts.shape).astype(np.float32)

    M_perspective = cv2.getPerspectiveTransform(src_pts, dst_pts)

    # Expandir M_combined de 2x3 para 3x3
    M_affine_homog = np.vstack([M_combined, [0, 0, 1]])

    # Combinar tudo (afim + perspectiva)
    M_total = M_perspective @ M_affine_homog

    # Aplicar transformação completa
    aug = cv2.warpPerspective(
        image, M_total, (w, h),
        borderValue=(255, 255, 255)
    )

    augmented_images.append(aug)

# Mostrar as imagens geradas
plt.figure(figsize=(16, 8))
for idx, img in enumerate(augmented_images):
    plt.subplot(2, 4, idx+1)
    plt.imshow(img)
    plt.title(f'Augment {idx+1}')
    plt.axis('off')

plt.tight_layout()
plt.show()
